In [7]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from io import StringIO, BytesIO
import numpy as np
from datetime import datetime
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import time

# SurveyCTO Configuration
SURVEYCTO_USERNAME = "rachelle@reliefapplications.org"
SURVEYCTO_PASSWORD = "HELENKELLER2024!"

# SharePoint Configuration
SHAREPOINT_USERNAME = "rAssaf@HKI.org"
SHAREPOINT_PASSWORD = "R@ch^2627"
SHAREPOINT_SITE_URL = "https://hkw.sharepoint.com/teams/PBI_MER_Data"
SHAREPOINT_FOLDER_NAME = "IM_Supervision_Harmonized_Data"

# IM Supervision Dataset URLs
IM_SUPERVISION_ENDPOINTS = {
    "Burkina Faso": "https://keller2024.surveycto.com/api/v1/forms/data/csv/bf2024_im_supervsion_post_monitoring_21_06_24",
    "Côte d'Ivoire": "https://keller2024.surveycto.com/api/v1/forms/data/csv/ci2024_im_superviseur_r1_hki", 
    "Guinea": "https://keller2024.surveycto.com/api/v1/forms/data/csv/gn2024_supervision_SVA_round1",
    "Mali": "https://keller2024.surveycto.com/api/v1/forms/data/csv/ml2024_supervision_sva_sabine_round1", 
    "Niger": "https://keller2024.surveycto.com/api/v1/forms/data/csv/ne2024_im_superviseur_round1",
    "RDC": "https://keller2024.surveycto.com/api/v1/forms/data/csv/cd_2024_im_supervisor_r2",
    "Kenya": "https://keller2024.surveycto.com/api/v1/forms/data/csv/ke_2024_im_supervisor_r2_hki"
}

# IM Supervision Field Mapping Dictionary
IM_SUPERVISION_MAPPING = {
    "submission_date": {
        "Burkina Faso": "SubmissionDate",
        "Côte d'Ivoire": "SubmissionDate",
        "Guinea": "SubmissionDate",
        "Mali": "SubmissionDate",
        "Niger": "SubmissionDate",
        "Kenya": "SubmissionDate",
        "RDC": "SubmissionDate"
    },
    "start_time": {
        "Burkina Faso": "starttime",
        "Côte d'Ivoire": "starttime",
        "Guinea": "start",
        "Mali": "starttime",
        "Niger": "starttime",
        "Kenya": "starttime",
        "RDC": "starttime"
    },
    "end_time": {
        "Burkina Faso": "endtime",
        "Côte d'Ivoire": "endtime",
        "Guinea": "end",
        "Mali": "endtime",
        "Niger": "endtime",
        "Kenya": "endtime",
        "RDC": "endtime"
    },
    "today": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "today",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "device_id": {
        "Burkina Faso": "deviceid",
        "Côte d'Ivoire": "deviceid",
        "Guinea": "deviceid",
        "Mali": "deviceid",
        "Niger": "deviceid",
        "Kenya": "deviceid",
        "RDC": "deviceid"
    },
    "subscriber_id": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "subscriberid",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "sim_serial": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "simserial",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "device_phone_number": {
        "Burkina Faso": "devicephonenum",
        "Côte d'Ivoire": "devicephonenum",
        "Guinea": "phonenumber",
        "Mali": "devicephonenum",
        "Niger": "devicephonenum",
        "Kenya": "devicephonenum",
        "RDC": "devicephonenum"
    },
    "username": {
        "Burkina Faso": "username",
        "Côte d'Ivoire": "username",
        "Guinea": None,
        "Mali": "username",
        "Niger": "username",
        "Kenya": "username",
        "RDC": "username"
    },
    "device_info": {
        "Burkina Faso": "device_info",
        "Côte d'Ivoire": "device_info",
        "Guinea": None,
        "Mali": "device_info",
        "Niger": "device_info",
        "Kenya": "device_info",
        "RDC": "device_info"
    },
    "duration": {
        "Burkina Faso": "duration",
        "Côte d'Ivoire": "duration",
        "Guinea": None,
        "Mali": "duration",
        "Niger": "duration",
        "Kenya": "duration",
        "RDC": "duration"
    },
    "case_id": {
        "Burkina Faso": "caseid",
        "Côte d'Ivoire": "caseid",
        "Guinea": None,
        "Mali": "caseid",
        "Niger": "caseid",
        "Kenya": "caseid",
        "RDC": "caseid"
    },
    "country": {
        "Burkina Faso": "country",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": "country"
    },
    "year": {
        "Burkina Faso": "year",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": "year"
    },
    "round": {
        "Burkina Faso": "round",
        "Côte d'Ivoire": "round",
        "Guinea": None,
        "Mali": None,
        "Niger": "round",
        "Kenya": None,
        "RDC": "round"
    },
    "date_jour": {
        "Burkina Faso": "date_jour",
        "Côte d'Ivoire": "date_jour",
        "Guinea": None,
        "Mali": "date_jour",
        "Niger": "date_jour",
        "Kenya": "date_jour",
        "RDC": "date_jour"
    },
    "supervisor_id": {
        "Burkina Faso": "collect-superviseur",
        "Côte d'Ivoire": "collect-superviseur",
        "Guinea": "titre_superviseur",
        "Mali": "superviseur",
        "Niger": "monitorage-superviseur",
        "Kenya": "superviseur",
        "RDC": "collect-supervisor"
    },
    "monitor_id": {
        "Burkina Faso": "collect-moniteur",
        "Côte d'Ivoire": "collect-moniteur",
        "Guinea": "s1",
        "Mali": "moniteur",
        "Niger": "monitorage-moniteur",
        "Kenya": "moniteur",
        "RDC": "collect-team_lead"
    },
    "monitor_name": {
        "Burkina Faso": "collect-moniteur_name",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": "collect-enume_name"
    },
    "region": {
        "Burkina Faso": "collect-region",
        "Côte d'Ivoire": "collect-region",
        "Guinea": "region",
        "Mali": "region",
        "Niger": "monitorage-region",
        "Kenya": "region",
        "RDC": "collect-region"
    },
    "district": {
        "Burkina Faso": "collect-district",
        "Côte d'Ivoire": "collect-district",
        "Guinea": "district",
        "Mali": "district",
        "Niger": "monitorage-district",
        "Kenya": "district",
        "RDC": "collect-district"
    },
    "district_name": {
        "Burkina Faso": "collect-district_name",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "health_center": {
        "Burkina Faso": "collect-centre_sante",
        "Côte d'Ivoire": None,
        "Guinea": "cs",
        "Mali": "aire_sante",
        "Niger": "monitorage-aire_sante",
        "Kenya": None,
        "RDC": None
    },
    "sanitary_name": {
        "Burkina Faso": "collect-sanitaire_name",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "localite": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "localite",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "visit_date": {
        "Burkina Faso": "collect-jour",
        "Côte d'Ivoire": "collect-jour",
        "Guinea": "date",
        "Mali": "jour",
        "Niger": "monitorage-jour",
        "Kenya": "jour",
        "RDC": "collect-jour"
    },
    # Group 1 QS fields harmonized (same for all, just change prefixes as per country)
    "qs1": {
        "Burkina Faso": "collect-G1-QS1",
        "Côte d'Ivoire": "collect-G1-QS1",
        "Guinea": "q1",
        "Mali": "G1-QS1",
        "Niger": "monitorage-QS1",
        "Kenya": "G1-QS1",
        "RDC": "collect-G1-QS1"
    },
    "qs2": {
        "Burkina Faso": "collect-G1-QS2",
        "Côte d'Ivoire": "collect-G1-QS2",
        "Guinea": "q2",
        "Mali": "G1-QS2",
        "Niger": "monitorage-QS2",
        "Kenya": "G1-QS2",
        "RDC": "collect-G1-QS2"
    },
    "qs3": {
        "Burkina Faso": "collect-G1-QS3",
        "Côte d'Ivoire": "collect-G1-QS3",
        "Guinea": "q3",
        "Mali": "G1-QS3",
        "Niger": "monitorage-QS3",
        "Kenya": "G1-QS3",
        "RDC": "collect-G1-QS3"
    },
    "qs4": {
        "Burkina Faso": "collect-G1-QS4",
        "Côte d'Ivoire": "collect-G1-QS4",
        "Guinea": "q4",
        "Mali": "G1-QS4",
        "Niger": "monitorage-QS4",
        "Kenya": "G1-QS4",
        "RDC": "collect-G1-QS4"
    },
    "qs5": {
        "Burkina Faso": "collect-G1-QS5",
        "Côte d'Ivoire": "collect-G1-QS5",
        "Guinea": "q5",
        "Mali": "G1-QS5",
        "Niger": "monitorage-QS5",
        "Kenya": "G1-QS5",
        "RDC": "collect-G1-QS5"
    },
    "qs6": {
        "Burkina Faso": "collect-G1-QS6",
        "Côte d'Ivoire": "collect-G1-QS6",
        "Guinea": "q6",
        "Mali": "G1-QS6",
        "Niger": "monitorage-QS6",
        "Kenya": "G1-QS6",
        "RDC": "collect-G1-QS6"
    },
    "qs7": {
        "Burkina Faso": "collect-G1-QS7",
        "Côte d'Ivoire": "collect-G1-QS7",
        "Guinea": "q7",
        "Mali": "G1-QS7",
        "Niger": "monitorage-QS7",
        "Kenya": "G1-QS7",
        "RDC": "collect-G1-QS7"
    },
    "qs8": {
        "Burkina Faso": "collect-G1-QS8",
        "Côte d'Ivoire": "collect-G1-QS8",
        "Guinea": "q8",
        "Mali": "G1-QS8",
        "Niger": "monitorage-QS8",
        "Kenya": "G1-QS8",
        "RDC": "collect-G1-QS8"
    },
    "qs9": {
        "Burkina Faso": "collect-G1-QS9",
        "Côte d'Ivoire": "collect-G1-QS9",
        "Guinea": "q9",
        "Mali": "G1-QS9",
        "Niger": "monitorage-QS9",
        "Kenya": "G1-QS9",
        "RDC": "collect-G1-QS9"
    },
    "qs10": {
        "Burkina Faso": "collect-G1-QS10",
        "Côte d'Ivoire": "collect-G1-QS10",
        "Guinea": "q10",
        "Mali": "G1-QS10",
        "Niger": "monitorage-QS10",
        "Kenya": "G1-QS10",
        "RDC": "collect-G1-QS10"
    },
    "qs11": {
        "Burkina Faso": "collect-G1-QS11",
        "Côte d'Ivoire": "collect-G1-QS11",
        "Guinea": "q11",
        "Mali": "G1-QS11",
        "Niger": "monitorage-QS11",
        "Kenya": "G1-QS11",
        "RDC": "collect-G1-QS11"
    },
    "qs12": {
        "Burkina Faso": "collect-G1-QS12",
        "Côte d'Ivoire": "collect-G1-QS12",
        "Guinea": "q12",
        "Mali": "G1-QS12",
        "Niger": "monitorage-QS12",
        "Kenya": "G1-QS12",
        "RDC": "collect-G1-QS12"
    },
    "qs13": {
        "Burkina Faso": "collect-G1-QS13",
        "Côte d'Ivoire": "collect-G1-QS13",
        "Guinea": "q13",
        "Mali": "G1-QS13",
        "Niger": "monitorage-QS13",
        "Kenya": "G1-QS13",
        "RDC": "collect-G1-QS13"
    },
    "qs14": {
        "Burkina Faso": "collect-G1-QS14",
        "Côte d'Ivoire": "collect-G1-QS14",
        "Guinea": "q14",
        "Mali": "G1-QS14",
        "Niger": "monitorage-QS14",
        "Kenya": "G1-QS14",
        "RDC": "collect-G1-QS14"
    },
    # Special and age-grouped QS15, QS16, QS17, QS18 fields
    "qs15": {
        "Burkina Faso": None,
        "Côte d'Ivoire": "collect-G1-QS15",
        "Guinea": None,
        "Mali": "G1-QS15",
        "Niger": None,
        "Kenya": "G1-QS15",
        "RDC": None
    },
    "qs15_6_11": {
        "Burkina Faso": "collect-G1-QS15_6_11",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": "monitorage-QS15_1",
        "Kenya": None,
        "RDC": "collect-G1-QS15_6_11"
    },
    "qs15_12_59": {
        "Burkina Faso": "collect-G1-QS15_12_59",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": "monitorage-QS15_3",
        "Kenya": None,
        "RDC": "collect-G1-QS15_12_59"
    },
    "qs16_1": {
        "Burkina Faso": "collect-G1-QS16_1",
        "Côte d'Ivoire": "collect-G1-QS16_1",
        "Guinea": None,
        "Mali": "G1-QS16_1",
        "Niger": "monitorage-QS16_1",
        "Kenya": "G1-QS16_1",
        "RDC": "collect-G1-QS16_1"
    },
    "qs16_2": {
        "Burkina Faso": "collect-G1-QS16_2",
        "Côte d'Ivoire": "collect-G1-QS16_2",
        "Guinea": None,
        "Mali": "G1-QS16_2",
        "Niger": "monitorage-QS16_2",
        "Kenya": "G1-QS16_2",
        "RDC": "collect-G1-QS16_2"
    },
    "qs16_3": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": "monitorage-QS16_3",
        "Kenya": None,
        "RDC": None
    },
    "qs17_1": {
        "Burkina Faso": "collect-G1-QS17_1",
        "Côte d'Ivoire": "collect-G1-QS17_1",
        "Guinea": None,
        "Mali": "G1-QS17_1",
        "Niger": "monitorage-QS17_1",
        "Kenya": "G1-QS17_1",
        "RDC": "collect-G1-QS17_1"
    },
    "qs17_2": {
        "Burkina Faso": "collect-G1-QS17_2",
        "Côte d'Ivoire": "collect-G1-QS17_2",
        "Guinea": None,
        "Mali": "G1-QS17_2",
        "Niger": "monitorage-QS17_2",
        "Kenya": "G1-QS17_2",
        "RDC": "collect-G1-QS17_2"
    },
    "qs17_3": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": "monitorage-QS17_3",
        "Kenya": None,
        "RDC": None
    },
    "qs18_1": {
        "Burkina Faso": "collect-G1-QS18_1",
        "Côte d'Ivoire": "collect-G1-QS18_1",
        "Guinea": None,
        "Mali": "G1-QS18_1",
        "Niger": "monitorage-QS18_1",
        "Kenya": "G1-QS18_1",
        "RDC": "collect-G1-QS18_1"
    },
    "qs18_2": {
        "Burkina Faso": "collect-G1-QS18_2",
        "Côte d'Ivoire": "collect-G1-QS18_2",
        "Guinea": None,
        "Mali": "G1-QS18_2",
        "Niger": "monitorage-QS18_2",
        "Kenya": "G1-QS18_2",
        "RDC": "collect-G1-QS18_2"
    },
    "qs18_3": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": "monitorage-QS18_3",
        "Kenya": None,
        "RDC": None
    },
    "qs19": {
        "Burkina Faso": "collect-G1-QS19",
        "Côte d'Ivoire": "collect-G1-QS19",
        "Guinea": "q18",
        "Mali": "G1-QS19",
        "Niger": "monitorage-QS19",
        "Kenya": "G1-QS19",
        "RDC": "collect-G1-QS19"
    },
    "qs20": {
        "Burkina Faso": "collect-G1-QS20",
        "Côte d'Ivoire": "collect-G1-QS20",
        "Guinea": None,
        "Mali": "G1-QS20",
        "Niger": "monitorage-QS20",
        "Kenya": "G1-QS20",
        "RDC": "collect-G1-QS20"
    },
    "qs20_o": {
        "Burkina Faso": "collect-G1-QS20_o",
        "Côte d'Ivoire": "collect-G1-QS20_o",
        "Guinea": None,
        "Mali": "G1-QS20_o",
        "Niger": "monitorage-QS20_o",
        "Kenya": "G1-QS20_o",
        "RDC": "collect-G1-QS20_o"
    },
    # Guinea unique fields
    "q16": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q16",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q18": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q18",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q21": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q21",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q23a": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q23a",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q23b": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q23b",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q25": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q25",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q27": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q27",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q28": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q28",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "q42": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q42",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    # GPS/coordinates
    "gps_latitude": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q43-Latitude",
        "Mali": "G1-mode_gps-Latitude",
        "Niger": "monitorage-coord-Latitude",
        "Kenya": None,
        "RDC": None
    },
    "gps_longitude": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q43-Longitude",
        "Mali": "G1-mode_gps-Longitude",
        "Niger": "monitorage-coord-Longitude",
        "Kenya": None,
        "RDC": None
    },
    "gps_altitude": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q43-Altitude",
        "Mali": "G1-mode_gps-Altitude",
        "Niger": "monitorage-coord-Altitude",
        "Kenya": None,
        "RDC": None
    },
    "gps_accuracy": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "q43-Accuracy",
        "Mali": "G1-mode_gps-Accuracy",
        "Niger": "monitorage-coord-Accuracy",
        "Kenya": None,
        "RDC": None
    },
    "certification": {
        "Burkina Faso": None,
        "Côte d'Ivoire": None,
        "Guinea": "Certification",
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    # Meta
    "meta_instance_id": {
        "Burkina Faso": "meta-instanceID",
        "Côte d'Ivoire": "meta-instanceID",
        "Guinea": "meta-instanceID",
        "Mali": "meta-instanceID",
        "Niger": "meta-instanceID",
        "Kenya": "meta-instanceID",
        "RDC": "meta-instanceID"
    },
    "meta_instance_name": {
        "Burkina Faso": "meta-instanceName",
        "Côte d'Ivoire": None,
        "Guinea": None,
        "Mali": None,
        "Niger": None,
        "Kenya": None,
        "RDC": None
    },
    "form_version": {
        "Burkina Faso": "formdef_version",
        "Côte d'Ivoire": "formdef_version",
        "Guinea": "formdef_version",
        "Mali": "formdef_version",
        "Niger": "formdef_version",
        "Kenya": "formdef_version",
        "RDC": "formdef_version"
    },
    "review_quality": {
        "Burkina Faso": "review_quality",
        "Côte d'Ivoire": "review_quality",
        "Guinea": "review_quality",
        "Mali": "review_quality",
        "Niger": "review_quality",
        "Kenya": "review_quality",
        "RDC": "review_quality"
    },
    "key": {
        "Burkina Faso": "KEY",
        "Côte d'Ivoire": "KEY",
        "Guinea": "KEY",
        "Mali": "KEY",
        "Niger": "KEY",
        "Kenya": "KEY",
        "RDC": "KEY"
    }
}

def load_country_data(country, url):
    """Load data for a single country from SurveyCTO"""
    try:
        response = requests.get(url, auth=HTTPBasicAuth(SURVEYCTO_USERNAME, SURVEYCTO_PASSWORD))
        response.raise_for_status()
        data = pd.read_csv(StringIO(response.text), low_memory=False)
        
        print(f"✓ {country}: {len(data)} rows loaded")
        return data
    
    except Exception as e:
        print(f"✗ {country}: Error - {str(e)}")
        return None

def harmonize_columns(data, country):
    """Apply IM_SUPERVISION_MAPPING to harmonize column names"""
    harmonized_data = data.copy()
    
    # Create reverse mapping for this country
    column_mapping = {}
    for std_name, country_mapping in IM_SUPERVISION_MAPPING.items():
        if country in country_mapping:
            original_col = country_mapping[country]
            if original_col and original_col in data.columns:
                column_mapping[original_col] = std_name
    
    # Rename columns
    harmonized_data = harmonized_data.rename(columns=column_mapping)
    
    # Add country identifier
    harmonized_data['Country'] = country
    
    print(f"  → {country}: {len(column_mapping)} columns harmonized")
    
    return harmonized_data

def safely_parse_date(date_str, formats):
    """Try multiple date formats to parse a date string"""
    if pd.isna(date_str) or date_str == '':
        return pd.NaT
        
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except (ValueError, TypeError):
            continue
    
    # If none of the formats work, try the flexible parser as last resort
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

def prepare_for_analysis(df):
    """Prepare harmonized data for analysis"""
    if df.empty:
        return df
        
    # Define common date formats
    submission_date_formats = [
        '%b %d, %Y %I:%M:%S %p',  # Jun 22, 2024 6:22:04 PM
        '%b %d, %Y %H:%M:%S',      # Jun 22, 2024 18:22:04
        '%Y-%m-%d %H:%M:%S'        # 2024-06-22 18:22:04
    ]
    
    simple_date_formats = [
        '%Y-%m-%d',                # 2024-06-22
        '%b %d, %Y',               # Jun 22, 2024
        '%d-%m-%Y',                # 22-06-2024
        '%Y/%m/%d'                 # 2024/06/22
    ]
    
    # Convert date columns with appropriate formats
    if 'submission_date' in df.columns:
        df['submission_date'] = df['submission_date'].apply(
            lambda x: safely_parse_date(x, submission_date_formats)
        )
    
    # Handle simple date columns
    for col in ['interview_date', 'visit_date', 'QS18_start_date', 'QS18_end_date', 'QS18_next_date']:
        if col in df.columns:
            df[col] = df[col].apply(
                lambda x: safely_parse_date(x, simple_date_formats)
            )
    
    # Create calculated date fields
    if 'submission_date' in df.columns:
        # Extract date components safely
        df['submission_year'] = df['submission_date'].dt.year
        df['submission_month'] = df['submission_date'].dt.month
        df['submission_month_name'] = df['submission_date'].dt.strftime('%B')
        df['submission_quarter'] = df['submission_date'].dt.quarter
        
        # Use a try-except block for the week calculation
        try:
            df['submission_week'] = df['submission_date'].dt.isocalendar().week
        except:
            df['submission_week'] = df['submission_date'].dt.week
    
    # Standardize numeric columns
    numeric_columns = [
        'QS1_distribution_methods', 'QS2_frequency_methods', 'QS5_revisit_count',
        'QS15_target_6_11', 'QS15_target_12_59', 
        'QS16_registered_6_11', 'QS16_registered_12_59', 'QS16_registered_other',
        'QS17_received_6_11', 'QS17_received_12_59', 'QS17_received_other'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Create coverage rate columns
    if 'QS16_registered_6_11' in df.columns and 'QS17_received_6_11' in df.columns:
        valid_mask = (df['QS16_registered_6_11'] > 0) & df['QS16_registered_6_11'].notna()
        df.loc[valid_mask, 'coverage_rate_6_11'] = (df.loc[valid_mask, 'QS17_received_6_11'] / 
                                                  df.loc[valid_mask, 'QS16_registered_6_11'] * 100).round(2)
        
    if 'QS16_registered_12_59' in df.columns and 'QS17_received_12_59' in df.columns:
        valid_mask = (df['QS16_registered_12_59'] > 0) & df['QS16_registered_12_59'].notna()
        df.loc[valid_mask, 'coverage_rate_12_59'] = (df.loc[valid_mask, 'QS17_received_12_59'] / 
                                                   df.loc[valid_mask, 'QS16_registered_12_59'] * 100).round(2)
        
    if all(col in df.columns for col in ['QS16_registered_6_11', 'QS16_registered_12_59', 
                                         'QS17_received_6_11', 'QS17_received_12_59']):
        total_registered = df['QS16_registered_6_11'].fillna(0) + df['QS16_registered_12_59'].fillna(0)
        total_received = df['QS17_received_6_11'].fillna(0) + df['QS17_received_12_59'].fillna(0)
        
        valid_mask = (total_registered > 0)
        df.loc[valid_mask, 'overall_coverage_rate'] = (total_received[valid_mask] / 
                                                     total_registered[valid_mask] * 100).round(2)
    
    # Handle missing values in text columns
    text_columns = [
        'QS12_challenges', 'QS13_successes', 'QS14_recommendations', 
        'QS20_other_issues_text'
    ]
    
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna('')
    
    # Add data extraction timestamp
    df['data_extraction_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return df

def upload_to_sharepoint(file_content, filename, doc_lib_name="Documents"):
    """Upload file to SharePoint"""
    library_names = [doc_lib_name, "Shared Documents", "Documents", "Shared%20Documents"]
    
    for lib_name in library_names:
        try:
            print(f"  📤 Trying library '{lib_name}' for {filename}...")
            
            ctx = ClientContext(SHAREPOINT_SITE_URL).with_credentials(
                UserCredential(SHAREPOINT_USERNAME, SHAREPOINT_PASSWORD)
            )
            
            doc_lib = ctx.web.lists.get_by_title(lib_name)
            ctx.load(doc_lib)
            ctx.execute_query()
            
            root_folder = doc_lib.root_folder
            ctx.load(root_folder)
            ctx.execute_query()
            
            # Create or access folder
            try:
                target_folder = root_folder.folders.get_by_url(SHAREPOINT_FOLDER_NAME)
                ctx.load(target_folder)
                ctx.execute_query()
            except:
                target_folder = root_folder.folders.add(SHAREPOINT_FOLDER_NAME)
                ctx.execute_query()
                print(f"  📁 Created folder: {SHAREPOINT_FOLDER_NAME}")
            
            # Delete existing file
            try:
                existing_file = target_folder.files.get_by_url(filename)
                existing_file.delete_object()
                ctx.execute_query()
                print(f"  🗑️ Deleted existing {filename}")
            except:
                pass
            
            # Upload new file
            target_folder.upload_file(filename, file_content)
            ctx.execute_query()
            print(f"  ✅ Successfully uploaded {filename} to '{lib_name}'")
            return True
            
        except Exception as e:
            print(f"  ⚠️ Failed with library '{lib_name}': {str(e)}")
            continue
    
    print(f"  ❌ Failed to upload {filename} to any library")
    return False

def upload_with_retry(file_content, filename, max_retries=3):
    """Upload with retry mechanism"""
    for attempt in range(max_retries):
        print(f"  🔄 Upload attempt {attempt + 1}/{max_retries}")
        
        if upload_to_sharepoint(file_content, filename):
            return True
        
        if attempt < max_retries - 1:
            print(f"  ⏳ Waiting 5 seconds before retry...")
            time.sleep(5)
    
    return False

def test_sharepoint_connection():
    """Test SharePoint connection"""
    try:
        print("🔍 Testing SharePoint connection...")
        
        ctx = ClientContext(SHAREPOINT_SITE_URL).with_credentials(
            UserCredential(SHAREPOINT_USERNAME, SHAREPOINT_PASSWORD)
        )
        
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        print(f"✅ Connected to site: {web.title}")
        return True
        
    except Exception as e:
        print(f"❌ Connection failed: {str(e)}")
        return False

def create_im_supervision_harmonized():
    """Create harmonized IM Supervision table"""
    
    print("🚀 IM SUPERVISION DATA HARMONIZATION")
    print("=" * 60)
    
    # Test SharePoint connection
    if not test_sharepoint_connection():
        print("⚠️ SharePoint connection failed - will skip upload")
    
    all_country_data = []
    
    # Load data for each country
    for country, url in IM_SUPERVISION_ENDPOINTS.items():
        print(f"\n📥 Loading {country}...")
        
        # Load raw data
        raw_data = load_country_data(country, url)
        
        if raw_data is not None:
            # Apply harmonization mapping
            harmonized_data = harmonize_columns(raw_data, country)
            all_country_data.append(harmonized_data)
    
    if not all_country_data:
        print("❌ No data loaded!")
        return None
    
    # Combine all countries
    print(f"\n🔗 Combining data from {len(all_country_data)} countries...")
    combined_data = pd.concat(all_country_data, ignore_index=True, sort=False)
    
    # Prepare data for analysis
    print(f"🔄 Preparing data for analysis...")
    final_data = prepare_for_analysis(combined_data)
    
    # Summary
    print(f"\n" + "=" * 60)
    print("IM SUPERVISION - HARMONIZED SUMMARY")
    print("=" * 60)
    print(f"Total rows: {len(final_data):,}")
    print(f"Total columns: {len(final_data.columns)}")
    print(f"Countries: {final_data['Country'].nunique()}")
    
    # Show harmonized columns with country counts
    harmonized_cols = [col for col in IM_SUPERVISION_MAPPING.keys() if col in final_data.columns]
    print(f"Harmonized columns: {len(harmonized_cols)}")
    
    print(f"\nCountry breakdown:")
    country_counts = final_data['Country'].value_counts().sort_index()
    for country, count in country_counts.items():
        print(f"  {country:15}: {count:6,} rows")
    
    # Upload to SharePoint
    print(f"\n📤 Uploading to SharePoint...")
    
    filename = "IM_Supervision_Harmonized.csv"
    csv_buffer = BytesIO()
    final_data.to_csv(csv_buffer, index=False, encoding='utf-8')
    csv_buffer.seek(0)
    
    if upload_with_retry(csv_buffer.getvalue(), filename):
        print(f"\n🎉 SUCCESS! IM Supervision harmonized data uploaded to SharePoint")
        print(f"📁 Location: {SHAREPOINT_FOLDER_NAME}/{filename}")
        print(f"📊 Dataset: {len(final_data):,} records")
    else:
        print(f"\n⚠️ Upload failed - but harmonization was successful")
    
    return final_data

# Run the harmonization
if __name__ == "__main__":
    # Create harmonized table
    im_supervision_harmonized = create_im_supervision_harmonized()
    
    # Display sample data
    if im_supervision_harmonized is not None:
        print(f"\n🎉 IM Supervision harmonized table is ready!")
        print(f"Shape: {im_supervision_harmonized.shape}")
        
        print(f"\n" + "=" * 80)
        print("SAMPLE DATA (First 5 rows):")
        print("=" * 80)
        print(im_supervision_harmonized.head())
        
        # Show key harmonized columns
        key_columns = ['Country', 'region', 'district', 'submission_date', 
                      'QS16_registered_6_11', 'QS17_received_6_11', 'coverage_rate_6_11']
        available_key_cols = [col for col in key_columns if col in im_supervision_harmonized.columns]
        
        if available_key_cols:
            print(f"\n" + "=" * 80)
            print("KEY HARMONIZED COLUMNS SAMPLE:")
            print("=" * 80)
            print(im_supervision_harmonized[available_key_cols].head())
    
    print(f"\n✨ Harmonization complete! Variable 'im_supervision_harmonized' is ready for analysis.")

🚀 IM SUPERVISION DATA HARMONIZATION
🔍 Testing SharePoint connection...
✅ Connected to site: PBI_MER_Data

📥 Loading Burkina Faso...
✓ Burkina Faso: 36 rows loaded
  → Burkina Faso: 52 columns harmonized

📥 Loading Côte d'Ivoire...
✓ Côte d'Ivoire: 220 rows loaded
  → Côte d'Ivoire: 42 columns harmonized

📥 Loading Guinea...
✓ Guinea: 53 rows loaded
  → Guinea: 47 columns harmonized

📥 Loading Mali...
✓ Mali: 206 rows loaded
  → Mali: 48 columns harmonized

📥 Loading Niger...
✓ Niger: 74 rows loaded
  → Niger: 53 columns harmonized

📥 Loading RDC...
✓ RDC: 131 rows loaded
  → RDC: 48 columns harmonized

📥 Loading Kenya...
✓ Kenya: 26 rows loaded
  → Kenya: 43 columns harmonized

🔗 Combining data from 7 countries...
🔄 Preparing data for analysis...

IM SUPERVISION - HARMONIZED SUMMARY
Total rows: 746
Total columns: 83
Countries: 7
Harmonized columns: 74

Country breakdown:
  Burkina Faso   :     36 rows
  Côte d'Ivoire  :    220 rows
  Guinea         :     53 rows
  Kenya          :     